
Попытка сделать трекер видео, используя yolov5

###Download MOT17 data and labels

In [ ]:
# Full data splitted by frames
# Really  we don't need it for this task but some images will be used for tests
!wget https://motchallenge.net/data/MOT17Det.zip
!unzip MOT17Det.zip

# Ground true bounding box
!wget https://motchallenge.net/data/MOT17Labels.zip
!unzip MOT17Labels.zip

# File with wideo
# Warning video resolution in this copy smaller than original video
!wget https://motchallenge.net/sequenceVideos/MOT17-09-SDP-raw.webm

in_video = 'content/MOT17-09-SDP-raw.webm'

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/MOT17-02/img1/000451.jpg  
  inflating: train/MOT17-02/img1/000452.jpg  
  inflating: train/MOT17-02/img1/000453.jpg  
  inflating: train/MOT17-02/img1/000454.jpg  
  inflating: train/MOT17-02/img1/000455.jpg  
  inflating: train/MOT17-02/img1/000456.jpg  
  inflating: train/MOT17-02/img1/000457.jpg  
  inflating: train/MOT17-02/img1/000458.jpg  
  inflating: train/MOT17-02/img1/000459.jpg  
  inflating: train/MOT17-02/img1/000460.jpg  
  inflating: train/MOT17-02/img1/000461.jpg  
  inflating: train/MOT17-02/img1/000462.jpg  
  inflating: train/MOT17-02/img1/000463.jpg  
  inflating: train/MOT17-02/img1/000464.jpg  
  inflating: train/MOT17-02/img1/000465.jpg  
  inflating: train/MOT17-02/img1/000466.jpg  
  inflating: train/MOT17-02/img1/000467.jpg  
  inflating: train/MOT17-02/img1/000468.jpg  
  inflating: train/MOT17-02/img1/000469.jpg  
  inflating: train/MOT17-02/img1/000470.jpg  
  inflating:

### Look at GT labels

All frame numbers, target IDs and bounding boxes are 1-based. World coordinates x,y,z are ignored for the 2D challenge


In [ ]:
import pandas as pd
#https://github.com/dendorferpatrick/MOTChallengeEvalKit/tree/master/MOT
gt = '/content/train/MOT17-09-SDP/gt/gt.txt'
labels = pd.read_csv(gt,
                     sep=',',
                     names=["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "conf", "x", "y", "z"])

print(labels.iloc[[2]])
labels.head()

   frame  id  bb_left  bb_top  bb_width  bb_height  conf  x    y   z
2      3   1      264     449       102        263     1  1  1.0 NaN


,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,1,260,450,102,262,1,1,1.0,NaN
1,2,1,262,449,102,263,1,1,1.0,NaN
2,3,1,264,449,102,263,1,1,1.0,NaN
3,4,1,266,448,102,264,1,1,1.0,NaN
4,5,1,268,448,102,264,1,1,1.0,NaN


### Create person detector based on YOLO model

In [ ]:
import torch

class PersonDetector:
    def __init__(self, model_name="yolov5s"):
        self.model = torch.hub.load("ultralytics/yolov5", model_name, pretrained=True)
        self.model.conf = 0.5
        self.model.classes = [0]

    def __call__(self, imgs):
        results = self.model(imgs)
        out = []

        for result in results.xyxy:
            bboxes = result[:, :4].cpu().numpy()
            out.append(bboxes)

        return out

### Smoke test for your PersonDetector

In [ ]:
detector = PersonDetector()
imgs = ['/content/train/MOT17-09/img1/000001.jpg','/content/train/MOT17-09/img1/000001.jpg']
bboxes_with_persons = detector(imgs)
print("Persons",bboxes_with_persons)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-3 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Persons [array([[     1702.5,      388.08,      1849.9,      726.85],
       [     255.37,         455,      353.55,       708.6],
       [     1250.3,      539.38,      1309.1,      656.48],
       [     1290.1,      462.35,      1359.8,      655.07],
       [          0,      327.29,      90.833,       902.8],
       [     114.93,       499.8,       202.2,      743.33],
       [     1883.3,      382.04,      1919.7,      577.44]], dtype=float32), array([[     1702.5,      388.08,      1849.9,      726.85],
       [     255.37,         455,      353.55,       708.6],
       [     1250.3,      539.38,      1309.1,      656.48],
       [     1290.1,      462.35,      1359.8,      655.07],
       [          0,      327.29,      90.833,       902.8],
       [     114.93,       499.8,       202.2,      743.33],
       [     1883.3,      382.04,      1919.7,      577.44]], dtype=float32)]


### Clone NORFAIR
https://github.com/tryolabs/norfair


or [SORT](https://github.com/abewley/sort.git)
but it repo is outdated :(



In [ ]:
# NORFAIR

!pip install norfair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.1 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=a35b212f27c8ce96566c108a7298f091fb007d75bbcbcaebfdefe1c147e06aca
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.20.1 requires rich>=1

You can use this code as a template:

https://github.com/Gan4x4/CV-HSE2019/blob/master/video/exercise.ipynb

In [ ]:
import os
print(os.listdir("train"))

['MOT17-02-SDP', 'MOT17-05-SDP', 'MOT17-02', 'MOT17-09-DPM', 'MOT17-13-SDP', 'MOT17-05', 'MOT17-11-FRCNN', 'MOT17-10-FRCNN', 'MOT17-09-FRCNN', 'MOT17-02-FRCNN', 'MOT17-10-SDP', 'MOT17-10', 'MOT17-09-SDP', 'MOT17-04-SDP', 'MOT17-02-DPM', 'MOT17-04-DPM', 'MOT17-04-FRCNN', 'MOT17-10-DPM', 'MOT17-05-FRCNN', 'MOT17-11', 'MOT17-13', 'MOT17-13-DPM', 'MOT17-09', 'MOT17-05-DPM', 'MOT17-13-FRCNN', 'MOT17-04', 'MOT17-11-SDP', 'MOT17-11-DPM']


In [ ]:
!unzip -o MOT17Labels.zip -d train

Archive:  MOT17Labels.zip
  inflating: train/test/MOT17-01-DPM/seqinfo.ini  
  inflating: train/test/MOT17-01-FRCNN/seqinfo.ini  
  inflating: train/test/MOT17-01-SDP/seqinfo.ini  
  inflating: train/test/MOT17-03-DPM/seqinfo.ini  
  inflating: train/test/MOT17-03-FRCNN/seqinfo.ini  
  inflating: train/test/MOT17-03-SDP/seqinfo.ini  
  inflating: train/test/MOT17-06-DPM/seqinfo.ini  
  inflating: train/test/MOT17-06-FRCNN/seqinfo.ini  
  inflating: train/test/MOT17-06-SDP/seqinfo.ini  
  inflating: train/test/MOT17-07-DPM/seqinfo.ini  
  inflating: train/test/MOT17-07-FRCNN/seqinfo.ini  
  inflating: train/test/MOT17-07-SDP/seqinfo.ini  
  inflating: train/test/MOT17-08-DPM/seqinfo.ini  
  inflating: train/test/MOT17-08-FRCNN/seqinfo.ini  
  inflating: train/test/MOT17-08-SDP/seqinfo.ini  
  inflating: train/test/MOT17-12-DPM/seqinfo.ini  
  inflating: train/test/MOT17-12-FRCNN/seqinfo.ini  
  inflating: train/test/MOT17-12-SDP/seqinfo.ini  
  inflating: train/test/MOT17-14-DPM/seqinfo

# ВАЖНО
Сделал  загрузку данных, аннотаций, инициализировал детектор. Попытался объединить детекции и трекер, но столкнулся с проблемой с форматом bounding boxes, поэтому не справился до конца, выполнено только частично.

In [ ]:
import cv2
import numpy as np
import os
import norfair

detector = PersonDetector()

video_path = "train/MOT17-09/img1"
gt_path = "train/MOT17-09/gt/gt.txt"

gt_data = np.loadtxt(gt_path, delimiter=',')

frame_files = sorted(os.listdir(video_path))
output_video = "output.mp4"

frame_sample = cv2.imread(os.path.join(video_path, frame_files[0]))
height, width, _ = frame_sample.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 30, (width, height))

tracker = norfair.Tracker(
    distance_function=norfair.distances.iou,
    distance_threshold=0.7
)

for frame_id, file in enumerate(frame_files, start=1):
    frame_path = os.path.join(video_path, file)
    frame = cv2.imread(frame_path)

    gt_boxes = gt_data[gt_data[:, 0] == frame_id][:, 2:6].astype(int)

    for x, y, w, h in gt_boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    bboxes = detector([frame_path])[0]

    detections = [
        norfair.Detection(
            points=np.array([[x, y], [x + w, y + h]]),
            scores=np.array([1.0, 1.0])
        )
        for x, y, w, h in np.array(bboxes)
    ]

    tracked_objects = tracker.update(detections=detections)

    for obj in tracked_objects:
        x1, y1, x2, y2 = map(int, obj.estimate)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f"ID {obj.id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    out.write(frame)

out.release()
print("Видео сохранено в output.mp4")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-3 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


AssertionError: Bounding boxes must be defined as np.array with (N, 4) shape, <norfair.tracker.Detection object at 0x78cd09bf77d0> given